In [24]:
# === Import Required Libraries === #
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import IsolationForest
import time
from tqdm import tqdm
import os

# === Get Current Working Directory (for file saving) === #
current_dir = os.getcwd()
print(current_dir)

# === Load Metadata File === #
# This CSV contains metadata for all active systems; we ensure STATION_ID is read as a string.
metadata_path = "meta_data_for_all_active_systems_for_calculation_20250401(CSV).csv"
metadata_df = pd.read_csv(metadata_path, dtype={"STATION_ID": str})  # Ensure STATION_ID is a string

# === Select First 840 Stations for Testing === #
filtered_metadata = metadata_df.head(840).copy()

# === Extract STATION_IDs for API Calls === #
site_ids = filtered_metadata["STATION_ID"].tolist()

# === Initialize Results DataFrame to Store Summaries === #
df_results = pd.DataFrame()

# === Start Processing Stations with Progress Bar === #
start_time = time.time()
with tqdm(total=len(site_ids), desc="Processing Stations", unit="station") as pbar:
    for i, DivrtID in enumerate(site_ids, start=1):
         # === Build API Request URL for Daily Discharge Data === #
        end_date = datetime.today().strftime("%Y-%m-%d")
        api_url = f"https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id={DivrtID}&end_date={end_date}&f=json"

        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                # === Create DataFrame from API JSON Response === #
                df = pd.DataFrame(data["data"], columns=["date", "value"])
                df.rename(columns={"date": "Date", "value": "DISCHARGE"}, inplace=True)
                df["STATION_ID"] = DivrtID

                # === Format Date and Assign Season === #
                df["Date"] = pd.to_datetime(df["Date"])
                df["SEASON"] = np.where(
                    df["Date"].dt.month.between(4, 9) | ((df["Date"].dt.month == 10) & (df["Date"].dt.day == 1)),
                    "Irrigation Season", "Non-Irrigation Season"
                )

                # === Convert Discharge Values to Numeric and Flag Basic Issues === #
                df["DISCHARGE"] = pd.to_numeric(df["DISCHARGE"], errors="coerce")
                
                # === Negative Values === #
                df["FLAG_NEGATIVE"] = df["DISCHARGE"] < 0
                
                # === Values Equal to Zero === #
                df["FLAG_ZERO"] = df["DISCHARGE"] == 0

                # === Filter Out Non-Positive Values for Statistical Tests === #
                df_nonzero = df[df["DISCHARGE"] > 0].copy()

                if not df_nonzero.empty:
                    # === Compute IQR Thresholds === #
                    Q1, Q3 = df_nonzero["DISCHARGE"].quantile([0.25, 0.75])
                    IQR = Q3 - Q1

                    # === Compute 90th Percentile Thresholds === #
                    discharge_90th_percentile = np.percentile(df_nonzero["DISCHARGE"].dropna(), 90)

                    # === Compute the 95th Percentile Thresholds === #
                    discharge_95th_percentile = np.percentile(df_nonzero["DISCHARGE"].dropna(), 95)
                    
                    # === Compute Rate of Change and Merge Back === #
                    df_nonzero["RATE_OF_CHANGE"] = df_nonzero["DISCHARGE"].diff().abs()
                    df = df.merge(df_nonzero[["Date", "RATE_OF_CHANGE"]], on="Date", how="left")

                    # === Flag Repeated Values (4 or more in a row) === #
                    df_nonzero["FLAG_REPEATED"] = df_nonzero["DISCHARGE"].groupby(
                        (df_nonzero["DISCHARGE"] != df_nonzero["DISCHARGE"].shift()).cumsum()
                    ).transform("count") >= 4

                    # === Run Isolation Forest for Outlier Detection === #
                    model = IsolationForest(contamination=0.05, random_state=42)
                    df_nonzero["OUTLIER_IF"] = model.fit_predict(df_nonzero[["DISCHARGE"]])
                    df_nonzero["OUTLIER_IF"] = df_nonzero["OUTLIER_IF"] == -1

                    # === Compute Relative Standard Deviation for Flagging Large Spikes === #
                    mean_discharge = df_nonzero["DISCHARGE"].mean()
                    df["PERCENT_DEV"] = ((df["DISCHARGE"] - mean_discharge).abs() / mean_discharge) * 100

                    # === Flag Discharges with Large Relative Deviation === #
                    threshold = 1000 # Over 1000% deviation
                    df["FLAG_RSD"] = (df["PERCENT_DEV"] > threshold) & (df["DISCHARGE"] != 0)

                    # === Merge Isolation Forest and Repeated Flags Back === #
                    df = df.merge(df_nonzero[["Date", "OUTLIER_IF", "FLAG_REPEATED"]], on="Date", how="left")
                else:
                    # === Fallback Values for Empty or Non-Positive Series === #
                    discharge_90th_percentile = 0
                    discharge_95th_percentile = 0
                    IQR = 0
                    Q1 = Q3 = 0
                    df["RATE_OF_CHANGE"] = np.nan
                    df["OUTLIER_IF"] = False
                    df["FLAG_REPEATED"] = False
                    df["PERCENT_DEV"] = np.nan
                    df["FLAG_RSD"] = False

                # === Flag Values > 95th Percentile Outliers, IQR Outliers Upper and Lower Bound, Flag Values Rate of Change > 90th Percentile Value  === #
                df["FLAG_Discharge"] = df["DISCHARGE"] > discharge_95th_percentile
                df["FLAG_IQR"] = (df["DISCHARGE"] < Q1 - 1.5 * IQR) | (df["DISCHARGE"] > Q3 + 1.5 * IQR)
                df["FLAG_RoC"] = df["RATE_OF_CHANGE"] > discharge_90th_percentile

                # === Above Max (Suspected Above Max)--only show common flagged values among the three methods (IQR, > 95th Percentile, IF) === #
                df["FLAG_ABOVE_MAX_OVERLAP"] = df["FLAG_IQR"] & df["FLAG_Discharge"] & df["OUTLIER_IF"]

                # === Large Spikes --only show common flagged values among the two methods (RSD,RoC) === #
                df["FLAG_LARGE_SPIKES"] = df["FLAG_RSD"] & df["FLAG_RoC"]

                # === Overall Flagged Record Indicator === #
                df["FLAGGED"] = df[
                    ["FLAG_NEGATIVE", "FLAG_ZERO", "FLAG_REPEATED", "FLAG_ABOVE_MAX_OVERLAP", "FLAG_LARGE_SPIKES"]
                ].any(axis=1)

                # === Create General Summary of Flags Per Station === #
                total_flagged = df["FLAGGED"].sum()
                total_not_flagged = len(df) - total_flagged

                station_summary = {
                    "STATION_ID": DivrtID,
                    "TOTAL_RECORDS": len(df),
                    "TOTAL_NEGATIVE": df["FLAG_NEGATIVE"].sum(),
                    "TOTAL_ZERO": df["FLAG_ZERO"].sum(),
                    "TOTAL_REPEATED": df["FLAG_REPEATED"].sum(),
                    "TOTAL_ABOVE_MAX": df["FLAG_ABOVE_MAX_OVERLAP"].sum(),
                    "TOTAL_LARGE_SPIKES": df["FLAG_LARGE_SPIKES"].sum(),
                    "TOTAL_FLAGGED": total_flagged,
                    "TOTAL_NOT_FLAGGED": total_not_flagged,
                
                    # Ratios based on total flagged
                    "TOTAL_NEGATIVE_RATIO": (df["FLAG_NEGATIVE"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "TOTAL_ZERO_RATIO": (df["FLAG_ZERO"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "TOTAL_REPEATED_RATIO": (df["FLAG_REPEATED"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "TOTAL_ABOVE_MAX_RATIO": (df["FLAG_ABOVE_MAX_OVERLAP"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "TOTAL_LARGE_SPIKES_RATIO": (df["FLAG_LARGE_SPIKES"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "TOTAL_FLAGGED_RATIO": (total_flagged / len(df)) * 100 if total_flagged else 0,
                    "TOTAL_NOT_FLAGGED_RATIO": (total_not_flagged / len(df)) * 100 if total_flagged else 0,
                }

                # === Seasonal Summaries === #
                df_ir = df[df["SEASON"] == "Irrigation Season"]
                df_nir = df[df["SEASON"] == "Non-Irrigation Season"]

                total_ir_flagged = df_ir["FLAGGED"].sum()
                total_nir_flagged = df_nir["FLAGGED"].sum()
                total_ir_not_flagged = len(df_ir) - total_ir_flagged
                total_nir_not_flagged = len(df_nir) - total_nir_flagged

                irrigation_season_summary = {
                    "STATION_ID": DivrtID,
                    "IR_RECORDS": len(df_ir),
                    "IR_NEGATIVE": df_ir["FLAG_NEGATIVE"].sum(),
                    "IR_ZERO": df_ir["FLAG_ZERO"].sum(),
                    "IR_REPEATED": df_ir["FLAG_REPEATED"].sum(),
                    "IR_ABOVE_MAX": df_ir["FLAG_ABOVE_MAX_OVERLAP"].sum(),
                    "IR_LARGE_SPIKES": df_ir["FLAG_LARGE_SPIKES"].sum(),
                    "IR_FLAGGED": total_ir_flagged,
                    "IR_NOT_FLAGGED": total_ir_not_flagged,
    
                    # Ratios based on total flagged
                    "NEGATIVE_IR_RATIO": (df_ir["FLAG_NEGATIVE"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "ZERO_IR_RATIO": (df_ir["FLAG_ZERO"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "REPEATED_IR_RATIO": (df_ir["FLAG_REPEATED"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "ABOVE_MAX_IR_RATIO": (df_ir["FLAG_ABOVE_MAX_OVERLAP"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "LARGE_SPIKES_IR_RATIO": (df_ir["FLAG_LARGE_SPIKES"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "FLAGGED_IR_RATIO": (total_ir_flagged / len(df_ir)) * 100 if total_ir_flagged else 0,
                    "NOT_FLAGGED_IR_RATIO": (total_ir_not_flagged / len(df_ir)) * 100 if total_ir_flagged else 0,
                }
                

                non_irrigation_season_summary = {
                    "STATION_ID": DivrtID,
                    "NIR_RECORDS": len(df_nir),
                    "NIR_NEGATIVE": df_nir["FLAG_NEGATIVE"].sum(),
                    "NIR_ZERO": df_nir["FLAG_ZERO"].sum(),
                    "NIR_REPEATED": df_nir["FLAG_REPEATED"].sum(),
                    "NIR_ABOVE_MAX": df_nir["FLAG_ABOVE_MAX_OVERLAP"].sum(),
                    "NIR_LARGE_SPIKES": df_nir["FLAG_LARGE_SPIKES"].sum(),
                    "NIR_FLAGGED": total_nir_flagged,
                    "NIR_NOT_FLAGGED": total_nir_not_flagged,
                
                    # Ratios based on total flagged
                    "NEGATIVE_NIR_RATIO": (df_nir["FLAG_NEGATIVE"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "ZERO_NIR_RATIO": (df_nir["FLAG_ZERO"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "REPEATED_NIR_RATIO": (df_nir["FLAG_REPEATED"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "ABOVE_MAX_NIR_RATIO": (df_nir["FLAG_ABOVE_MAX_OVERLAP"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "LARGE_SPIKES_NIR_RATIO": (df_nir["FLAG_LARGE_SPIKES"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "FLAGGED_NIR_RATIO": (total_nir_flagged / len(df_nir)) * 100 if total_nir_flagged else 0,
                    "NOT_FLAGGED_NIR_RATIO": (total_nir_not_flagged / len(df_nir)) * 100 if total_nir_flagged else 0,
                    }

                # Combine all summaries into one dictionary per station
                combined_summary = {
                    **station_summary,
                    **{k: v for k, v in irrigation_season_summary.items() if k != "STATION_ID"},
                    **{k: v for k, v in non_irrigation_season_summary.items() if k != "STATION_ID"}
                }
                
                # Append as one row
                df_results = pd.concat([df_results, pd.DataFrame([combined_summary])], ignore_index=True)

        pbar.update(1)

# Merge and export
merged_df = pd.merge(filtered_metadata, df_results, on="STATION_ID", how="left")
# === Add seaonality plot hyperlink === #
merged_df["SEASONALITY_WEBPAGE"] = (
    "https://flags-297769208259.us-west3.run.app/plot?id=" + merged_df["STATION_ID"].astype(str)
)
output_filename = "dvrt_data_station_seasonality_combined_overlap_stat_flagged_results_20250414.csv"
merged_df.to_csv(output_filename, index=False)

print(f"Final merged data saved as {output_filename}")

print ("Done")

C:\Users\pbenko\Documents\20250213_distribution_data_python\data\QC_Scripts\criteria_protocol\Multiple_station_csv_file_20250401


Processing Stations: 100%|██████████| 839/839 [08:56<00:00,  1.56station/s]

Final merged data saved as dvrt_data_station_seasonality_combined_overlap_stat_flagged_results_20250414.csv
Done
